In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
embeddings

'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: dffeb71e-503c-4d95-bbab-bb319decfdc0)')' thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config_sentence_transformers.json
Retrying in 1s [Retry 1/5].


HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
text="hey, good evening!"

embedding=embeddings.embed_query(text)


print(len(embedding))
print(embedding)

384
[-0.09392756968736649, 0.011261509731411934, 0.06838413327932358, 0.02725622057914734, -0.015564228408038616, -0.03390636667609215, 0.0756411999464035, 0.051016006618738174, -0.024912770837545395, -0.06779830157756805, -0.02861633151769638, 0.06608141213655472, 0.0019538227934390306, -0.0230705663561821, 0.0025448808446526527, 0.001895879046060145, -0.010382059961557388, -0.05011323466897011, -0.007470315787941217, 0.06947563588619232, -0.008353620767593384, -0.017505116760730743, 0.04110732302069664, 0.034253399819135666, -0.07099256664514542, -0.040894001722335815, 0.07672575116157532, 0.022262832149863243, -0.05778473615646362, -0.16383178532123566, -0.0804620236158371, 0.16319622099399567, -0.018692582845687866, 0.04609252139925957, 0.047315191477537155, 0.042730484157800674, -0.045380331575870514, -0.11161328107118607, 0.05568031966686249, 0.01813286915421486, 0.06276194751262665, 0.06870217621326447, -0.03963219374418259, 0.0315648652613163, -0.01168865617364645, 0.0378860831

sentences=[
    "hey, good morning! how are you?",
    "The cat is sitting on tree next to you",
    "the dog is looking at cat sitting on tree",
    "Tiger is dangerous animal",
    "lion is king of jungle",
]

In [31]:
sentences=[
    "Machine learning allows computers to learn from data.",
    "The cat slept on the warm sofa all afternoon.", 
    "Stock markets react quickly to changes in interest rates.",
    "A chef prepared a spicy meal using fresh ingredients.",
    "Deep learning is branch of artificial learning",
    "Lora and Qlora are parameter efficient fine tuning techniques"
]

In [32]:
sentence_embedding=embeddings.embed_documents(sentences)
print(sentence_embedding[0])
print(sentence_embedding[1])


[-0.020890841260552406, 0.017379703000187874, 0.07236635684967041, 0.010321949608623981, 0.031034335494041443, -0.01428581029176712, -0.020717956125736237, -0.08304940164089203, 0.024357829242944717, 0.01264865230768919, -0.05012471228837967, 0.05451709404587746, 0.031235339120030403, -0.06617546081542969, 0.04725773632526398, 0.037187498062849045, -0.017018573358654976, 0.009522483684122562, -0.0684274360537529, -0.10870412737131119, 0.030379386618733406, -0.042617302387952805, -0.06702607870101929, 0.023106658831238747, 0.002178577706217766, 0.09674712270498276, 0.01878809928894043, 0.054893024265766144, 0.01358052995055914, 0.00042933670920319855, 0.029117485508322716, -0.017038987949490547, -0.012888465076684952, 0.03748667985200882, -0.07866920530796051, 0.011858563870191574, -0.03940479829907417, 0.04321032762527466, 0.014334955252707005, -0.021447211503982544, -0.013328015804290771, -0.05593106523156166, 0.013494638726115227, 0.03647686913609505, 0.07630989700555801, 0.062602140

#### finding cosine similarity

In [33]:
import numpy as np


def cosine_similarity(vec1, vec2):
    dot_product=np.dot(vec1, vec2)
    mag_a=np.linalg.norm(vec1)
    mag_b=np.linalg.norm(vec2)
    return dot_product/(mag_a * mag_b)

In [34]:
for i in range(len(sentences)):
    for j in range(i+1, len(sentences)):
        similarity=cosine_similarity(sentence_embedding[i], sentence_embedding[j])
        print(f"'{sentences[i]}' vs '{sentences[j]}'")
        print(f"similarity===>{similarity:.3f}\n")

'Machine learning allows computers to learn from data.' vs 'The cat slept on the warm sofa all afternoon.'
similarity===>-0.039

'Machine learning allows computers to learn from data.' vs 'Stock markets react quickly to changes in interest rates.'
similarity===>0.156

'Machine learning allows computers to learn from data.' vs 'A chef prepared a spicy meal using fresh ingredients.'
similarity===>0.010

'Machine learning allows computers to learn from data.' vs 'Deep learning is branch of artificial learning'
similarity===>0.483

'Machine learning allows computers to learn from data.' vs 'Lora and Qlora are parameter efficient fine tuning techniques'
similarity===>0.068

'The cat slept on the warm sofa all afternoon.' vs 'Stock markets react quickly to changes in interest rates.'
similarity===>0.039

'The cat slept on the warm sofa all afternoon.' vs 'A chef prepared a spicy meal using fresh ingredients.'
similarity===>0.015

'The cat slept on the warm sofa all afternoon.' vs 'Deep learn

#### semantic search

In [35]:
query="what machine learning?"


In [36]:
def semantic_search(query,sentences,embeddings,top_k=3):
    query_embedding=embeddings.embed_query(query)
    document_embeddings=embeddings.embed_documents(sentences)

    similarities=[]

    for i, doc_emb in enumerate(document_embeddings):
        similarity=cosine_similarity(query_embedding, doc_emb)
        similarities.append((similarity, sentences[i]))
    similarities.sort(reverse=True)
    return similarities[:top_k]

In [37]:
results=semantic_search(query,sentences,embeddings)
results


[(np.float64(0.720865296802706),
  'Machine learning allows computers to learn from data.'),
 (np.float64(0.5304834967059504),
  'Deep learning is branch of artificial learning'),
 (np.float64(0.14088566978803574),
  'Lora and Qlora are parameter efficient fine tuning techniques')]